In [1]:
import os
import random
import skimage.data
from skimage import io
import skimage.transform
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2

In [2]:
label_dir = "/home/ysk/catkin_ws/laser_pic_code/pic/regression"
pic_dir = "/home/ysk/catkin_ws/laser_pic_code/pic/regression/picture"

train_txt = os.path.join(label_dir, "label.txt")
count = len(open(train_txt, 'r').readlines())
print(count)
file = open(train_txt, 'r')
s = file.readlines()
all_images = []
all_labels = []
for i in range(count-1):
    image1 = skimage.data.imread(os.path.join(pic_dir, s[i].split(";")[0]))
    label = s[i].split(";")[1:3]
    label = list(map(float,label))
    image2 = skimage.data.imread(os.path.join(pic_dir, s[i+1].split(";")[0]))
#     image3 = skimage.data.imread(os.path.join(pic_dir, s[i+2].split(";")[0]))
    img = cv2.merge([image1,image2])
#     img = cv2.merge([image1,image2,image3])

    all_images.append(img)
    all_labels.append(label)
    
print(np.shape(all_images))
print(np.shape(all_labels))

4026
(4025, 448, 448, 6)
(4025, 2)


In [3]:
def get_train_data_batch(num):
    idx = np.arange(0, count-1)
    np.random.shuffle(idx)
    data_idx = idx[:num]

    labels = []
    images = []

    for i in range(num):
        images.append(all_images[data_idx[i]])
        labels.append(all_labels[data_idx[i]])

    return np.asarray(images), np.asarray(labels)
X,Y = get_train_data_batch(10)
print(Y)
# for i in range(10):
#     Y[i][0] -= 17
#     Y[i][1] *= 10
# print(Y)

print(np.shape(X))
print(np.shape(Y))

[[ 19.04426547   0.81838358]
 [-13.83200505  -5.21575092]
 [ 15.37369764   0.71374568]
 [  4.22693601 -10.06561712]
 [ 30.78513453  -6.10967308]
 [ 12.59687405  -5.75057139]
 [-30.39005542   3.59023849]
 [  7.28538828   0.70559844]
 [-27.24415083  -7.41269307]
 [-28.83704038 -15.39850187]]
(10, 448, 448, 6)
(10, 2)


In [4]:
with tf.name_scope("input"):
    input_image = tf.placeholder("float", [None, 448,448,6],name='input_image')
#     input_image = tf.pad(image,np.array([[0, 0], [3, 3], [3, 3], [0, 0]]))
    y_ = tf.placeholder("float", [None, 2],name='y_') 

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")


#第一层卷积层
#参数的前两维为卷积核的维度，第三个参数为当前层的深度，第四个为输出到层的深度
W_conv1 = weight_variable([7, 7, 6, 64])
b_conv1 = bias_variable([64])

h_conv1 = tf.nn.relu(tf.nn.conv2d(input_image, W_conv1,strides=[1,2,2,1],padding="SAME") + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)   #输出变成112*112*64

#第二层卷积层
W_conv2 = weight_variable([3, 3, 64, 192])
b_conv2 = bias_variable([192])

h_conv2 = tf.nn.leaky_relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)  #输出变为56*56*192


#第三层卷积层
W_conv3 = weight_variable([3, 3, 192, 128])
b_conv3 = bias_variable([128])

h_conv3 = tf.nn.leaky_relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)   #输出为28*28*128


#第四层卷积层
W_conv4 = weight_variable([3, 3, 128, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.leaky_relu(conv2d(h_pool3, W_conv4) + b_conv4)
h_pool4 = max_pool_2x2(h_conv4)   #输出变为14*14*64


#密集连接层1
W_fc1 = weight_variable([14*14*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool4, [-1, 14*14*64])
h_fc1 = tf.nn.leaky_relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)   #输出为96*1024

keep_prob = tf.placeholder("float")
#relu函数作用是求出max(h_fc1,0)
h_fc1_drop1 = tf.nn.dropout(h_fc1, keep_prob)   #输出为96*1024


#密集连接层2
W_fc2 = weight_variable([1024, 512])
b_fc2 = bias_variable([512])

h_fc2 = tf.nn.leaky_relu(tf.matmul(h_fc1_drop1, W_fc2) + b_fc2)

#relu函数作用是求出max(h_fc1,0)
h_fc1_drop2 = tf.nn.dropout(h_fc2, keep_prob)

#输出层softmax层
W_fc3 = weight_variable([512, 2])

y_conv = tf.matmul(h_fc1_drop2, W_fc3)
# tf.summary.histogram('y_conv',y_conv)

# batch_x = images[1:3]
# batch_y = labels[1:3]
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     print("shape of outputs is:",sess.run(tf.shape(y_conv),feed_dict={input_image:batch_x, 
#                                                                            y_:batch_y,keep_prob:1.0}))
#         print("learning rate is :",sess.run(learning_rate,feed_dict = {global_step:i}))


starter_learning_rate = 0.001
steps_per_decay = 1000
decay_factor = 0.96
 
global_step = tf.Variable(tf.constant(0))
learning_rate = tf.train.exponential_decay(learning_rate = starter_learning_rate,
                                           global_step = global_step,
                                           decay_steps = steps_per_decay,
                                           decay_rate = decay_factor,
                                           staircase = True
                                           )

# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     for i in range(1000):
#         print("learning rate is :",sess.run(learning_rate,feed_dict = {global_step:i}))
  
delta=tf.constant(1.25)
#huber损失函数
loss = tf.reduce_mean(tf.multiply(tf.square(delta),
                                          tf.sqrt(1. + tf.square((y_ - y_conv)/delta)) - 1.))

# cross_entropy = tf.reduce_sum(tf.square(y_-y_conv)) 
tf.summary.scalar("loss",loss)

train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)
# train_step = tf.train.AdadeltaOptimizer(learning_rate).minimize(cross_entropy)
# train_step = tf.train.RMSPropOptimizer(learning_rate).minimize(cross_entropy)

# correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# tf.summary.scalar("accuracy", accuracy)
merged = tf.summary.merge_all()

batch_size = 4                
checkpointsPath = "./checkpoints/"
reload = False

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #保存训练好的模型一遍下次使用
    saver = tf.train.Saver()
    writer = tf.summary.FileWriter("logs_dir/", sess.graph)
    
    if not os.path.exists(checkpointsPath):
        os.mkdir(checkpointsPath)
    
    if reload:
        checkPoint = tf.train.get_checkpoint_state(checkpointsPath)
        if checkPoint and checkPoint.model_checkpoint_path:
            saver.restore(sess,checkPoint.model_checkpoint_path)
            print("restored %s" % checkPoint.model_checkpoint_path)
        else:
            print("no checkpoint found!")
        
    for i in tqdm(range(200000)):
        batch_x,batch_y = get_train_data_batch(batch_size)
#         batch_y = batch_y-17.0
        batch_y = batch_y*100
        summary,_ = sess.run([merged,train_step],feed_dict={global_step:i,
                                                            input_image:batch_x, 
                                                            y_:batch_y, 
                                                            keep_prob:0.8})
        writer.add_summary(summary, i)
        if (i % 100) == 0 & (i != 0):
            print("save model")
            print("learning_rate is:\n",sess.run(learning_rate,feed_dict = {global_step:i}))
            print("real location is:\n",batch_y)
            print("prediction is:\n",sess.run(y_conv,feed_dict={input_image:batch_x, y_:batch_y,keep_prob:1.0}))
#             saver.save(sess, os.path.join(checkpointsPath,"saved_net.ckpt"), global_step=i)
            print("loss is:",sess.run(loss,feed_dict={input_image:batch_x, y_:batch_y, keep_prob:1.0}))
    print("done!")


  0%|          | 1/200000 [00:01<62:15:25,  1.12s/it]

save model
learning_rate is:
 0.001
real location is:
 [[ -873.93146787  -672.26224181]
 [ 1936.31901499  -916.49548219]
 [-2905.31843964 -2084.5693279 ]
 [-1590.2003584   -688.43409471]]
prediction is:
 [[-454205.75  785115.4 ]
 [-453068.3   776240.5 ]
 [-458052.38  771855.  ]
 [-450760.9   781284.94]]
loss is: 770547.2


  0%|          | 102/200000 [00:06<2:51:04, 19.48it/s]

save model
learning_rate is:
 0.001
real location is:
 [[-1591.26123371  -527.70529464]
 [-1326.9241685   -683.43332893]
 [-2368.76415133  -710.22633875]
 [ 3025.75943328    76.66482917]]
prediction is:
 [[-1.6660120e+03 -6.8695068e-01]
 [-2.4825134e+03 -3.3672491e+02]
 [-1.3342285e+03 -3.6329004e+02]
 [-9.2717822e+02 -4.0917352e+02]]
loss is: 1236.6147


  0%|          | 204/200000 [00:10<2:51:19, 19.44it/s]

...

save model
learning_rate is:
 2.9646503e-07
real location is:
 [[ 3109.39753435  -614.77278176]
 [ -884.61094074  -673.53171244]
 [-3060.72137364   767.48357779]
 [ 1256.51116604   -81.13447703]]
prediction is:
 [[ 3095.254    -615.3983 ]
 [ -870.61646  -670.5645 ]
 [-3032.337     773.3087 ]
 [ 1245.1304    -76.45109]]
loss is: 11.489492


100%|█████████▉| 199902/200000 [2:39:45<00:05, 19.58it/s]

save model
learning_rate is:
 2.9646503e-07
real location is:
 [[ 3090.25429316  -737.27371105]
 [-3111.93715946   906.08712708]
 [  457.05773128   -15.92605257]
 [ 2384.86164435  -895.47694931]]
prediction is:
 [[ 3063.5156    -739.0332  ]
 [-3082.6943     890.09924 ]
 [  429.8207     -18.804153]
 [ 2360.223     -891.28406 ]]
loss is: 19.325266


100%|██████████| 200000/200000 [2:39:50<00:00, 20.85it/s]

done!
